# Homework 4: Margin Maximization with SVM
In this homework, we will work towards implement a support vector machine.

In [44]:
import numpy as np
import math

# utility functions

def rv(x):
    """
    Converts given numpy array into row vector
    :param x: np 1d array or 2d vector array  
    """
    try:
        r, c = x.shape
        if r > 0:
            return x.T
        else:
            return x 
    except:
        return np.array([x])

def cv(x):
    """
    Converts given numpy array into column vector
    :param x: np 1d array or 2d vector array
    """
    try:
        r, c = x.shape
        if c > 0:
            return x.T
        else:
            return x 
    except:
        return np.array([x])

def magnitude(x):
    return np.linalg.norm(x)

def signed_dist(point, th, th0):
    return (th.T.dot(cv(point)) + th0)/magnitude(th)

def margin(point, label, th, th0):
    return label * signed_dist(point, th, th0)[0,0]



In [30]:
def sum_margin(data, labels, th, th0):
    """
    Sum up all the margins of the points in a dataset w.r.t a hyperplane

    :param data: dataset shape (i,N) where N is no. of points
    :param labels: labels corresponding to dataset with shape (1, N)
    :param th: parameters of the hyperplane with shape (1, i)
    :param th0: offset of hyperplane
    """
    total = 0
    for idx, point in enumerate(data.T):
        total += margin(point, labels[0, idx], th, th0)
    return total

def min_margin(data, labels, th, th0):
    """
    Get the minimum margin of the points in a dataset w.r.t a hyperplane
    
    :param data: dataset shape (i,N) where N is no. of points
    :param labels: labels corresponding to dataset with shape (1, N)
    :param th: parameters of the hyperplane with shape (1, i)
    :param th0: offset of hyperplane
    """
    mn = np.inf
    for idx, point in enumerate(data.T):
        mgn = margin(point, labels[0, idx], th, th0)
        mn = min(mn, mgn)
    return mn

def max_margin(data, labels, th, th0):
    """
    Get the maximum margin of the points in a dataset w.r.t a hyperplane
    
    :param data: dataset shape (i,N) where N is no. of points
    :param labels: labels corresponding to dataset with shape (1, N)
    :param th: parameters of the hyperplane with shape (1, i)
    :param th0: offset of hyperplane
    """
    mx = -np.inf
    for idx, point in enumerate(data.T):
        mgn = margin(point, labels[0, idx], th, th0)
        mx = max(mx, mgn)
    return mx

In [31]:
# data provided for homework for calculating min, max, sum of margin

data = np.array([[1, 2, 1, 2, 10, 10.3, 10.5, 10.7],
                 [1, 1, 2, 2,  2,  2,  2, 2]])
labels = np.array([[-1, -1, 1, 1, 1, 1, 1, 1]])
blue_th = np.array([[0, 1]]).T
blue_th0 = -1.5
red_th = np.array([[1, 0]]).T
red_th0 = -2.5

In [33]:
print("Margin sum, blue: ", sum_margin(data, labels, blue_th, blue_th0))
print("Margin min, blue: ", min_margin(data, labels, blue_th, blue_th0))
print("Margin max, blue: ", max_margin(data, labels, blue_th, blue_th0))
print("Margin sum, red: ", sum_margin(data, labels, red_th, red_th0))
print("Margin min, red: ", min_margin(data, labels, red_th, red_th0))
print("Margin max, red: ", max_margin(data, labels, red_th, red_th0))

Margin sum, blue:  4.0
Margin min, blue:  0.5
Margin max, blue:  0.5
Margin sum, red:  31.5
Margin min, red:  -1.5
Margin max, red:  8.2


# Hinge loss
Hinge loss is a loss function to maximize the minimum margin of a dataset to a hyperplane. Let:
- $\theta, \theta_0$: parameters for hyperplane
- $\gamma(x, y, \theta, \theta_0)$: margin of a point $x, y$ to hyperplane
- $\gamma_{\text{ref}}$: minimum margin acceptable

The equation for hinge loss is defined as:
$$
L_h\Big(\frac{\gamma(x,y,\theta, \theta_0)}{\gamma_{\text{ref}}}\Big) = \begin{cases} 1- \frac{\gamma(x,y,\theta, \theta_0)}{\gamma_{\text{ref}}} & \text{ if } \gamma(x, y, \theta, \theta_0) < \gamma_{\text{ref}} \\ 0 & \text{ otherwise}\end{cases}
$$

In [66]:
def hinge(v):
    return max(0, 1-v)

def hinge_loss(x,y, th, th0):
    """
    Calculate hinge loss for a point on a given hyperplane
    
    :param x: data of point
    :param y: label of point
    :param th: hyperplane parameters
    :param th0: hyperplane offset
    """
    mgn = margin(x, y, th, th0)
    return hinge(mgn)

In [40]:
# data provided for calculating hinge loss

data = np.array([[1.1, 1, 4],[3.1, 1, 2]])
labels = np.array([[1, -1, -1]])
th = np.array([[1, 1]]).T
th0 = -4

hinge_arr = []
for idx, data in enumerate(data.T):
    l = hinge_loss(data, labels[0, idx], th, th0, math.sqrt(2)/2)
    hinge_arr.append(l)
    
print(hinge_arr)


[0.7999999999999998, 0, 3.0]


# Implementing Gradient Descent
We will now implement a generic gradient descent function, gd, that has the following input arguments:

- f: a function whose input is an x, a column vector, and returns a scalar.
- df: a function whose input is an x, a column vector, and returns a column vector representing the gradient of f at x.
- x0: an initial value of x
- x, x0, which is a column vector.
- step_size_fn: a function that is given the iteration index (an integer) and returns a step size.
- max_iter: the number of iterations to perform

Our function gd returns a tuple:

- x: the value at the final step
- fs: the list of values of f found during all the iterations (including f(x0))
- xs: the list of values of x found during all the iterations (including x0)


In [41]:
def gd(f, df, x0, step_size_fn, max_iter):
    x_curr = x0
    fs = [f(x_curr)]
    xs = [x_curr]
    for i in range(max_iter):
        x_new = x_curr - df(x_curr)*step_size_fn(i)
        val_new = f(x_new)
        
        fs.append(val_new)
        xs.append(x_new)
        
        x_curr = x_new
    return (x_curr, fs, xs)

# Implementing Numerical Gradient 
We can compute the $i^{th}$ component of $\nabla f(x_0)$ through the following approximation:
$$
\frac{f(x_0 + \delta) - f(x_0 - \delta)}{2\delta} \text{ where }\delta>0
$$

Implement this as a function num_grad that takes as arguments the objective function f and a value of delta, and returns a new function that takes an x (a column vector of parameters) and returns a gradient column vector.

In [46]:
def num_grad(f, delta=0.001):
    def df(x):
        r, c = x.shape
        g = np.zeros((1,r))
        for idx in range(len(x)):
            d = np.zeros((1,r))
            d[0, idx] = delta
            gi = (f(x + d.T) - f(x - d.T))/(2*delta)
            g[0, idx] = gi
        return g.T    
    return df

# Calculating minimum of function
Write a function minimize that takes only a function f and uses this function and numerical gradient descent to return the local minimum. We have provided you with our implementations of num_grad and gd, so you should not redefine them in the code box below. You may use the default of delta=0.001 for num_grad.

In [ ]:
def minimize(f, x0, step_size_fn, max_iter):
    return gd(f, num_grad(f, delta=0.001), x0, step_size_fn, max_iter)[0]

# Objective Function of SVM
Implement the single-argument function hinge, which computes $L_h$, and use that to implement hinge loss for a data point and separator. Using the latter function, implement the SVM objective. Note that these functions should work for matrix/vector arguments, so that we can compute the objective for a whole dataset with one call. The objective function is defined as:
$$
J(\theta, \theta_0) = \frac{1}{n}\Big( \sum^n_{i=1} L_h(y^i(x^i\cdot\theta + \theta_0)) \Big ) + \lambda \Vert\theta\Vert^2
$$

In [49]:
# x is dxn, y is 1xn, th is dx1, th0 is 1x1, lam is a scalar
def svm_obj(x, y, th, th0, lam):
    d, n = x.shape
    sum_err = 0
    for idx, d in enumerate(x.T):
        label = y[0, idx]
        sum_err += hinge_loss(d, label, th, th0)
    training_err = sum_err/n
    return training_err + lam*magnitude(th)**2


# Gradient of individual parameters of SVM

In [135]:
# Returns the gradient of hinge(v) with respect to v.
def d_hinge(v):
    v[v < 1] = -1
    v[v >= 1] = 0
    return v

# Returns the gradient of hinge_loss(x, y, th, th0) with respect to th
def d_hinge_loss_th(x, y, th, th0):
    return d_hinge((x.T.dot(th) + th0)*y.T).T*y*x


# Returns the gradient of hinge_loss(x, y, th, th0) with respect to th0
def d_hinge_loss_th0(x, y, th, th0):
    return d_hinge((x.T.dot(th) + th0)*y.T).T*y

# Returns the gradient of svm_obj(x, y, th, th0) with respect to th
def d_svm_obj_th(x, y, th, th0, lam):
    d, n = x.shape
    d_hinge_loss_th_sum = np.sum(d_hinge_loss_th(x, y, th, th0), axis=1)
    return ((1/n)*d_hinge_loss_th_sum + 2*lam*th.T).T

# Returns the gradient of svm_obj(x, y, th, th0) with respect to th0
def d_svm_obj_th0(x, y, th, th0, lam):
    d, n = x.shape
    d_hinge_loss_th0_sum = np.array([np.sum(d_hinge_loss_th0(x, y, th, th0), axis=1)]).T
    return (1/n)*d_hinge_loss_th0_sum

# Returns the full gradient as a single vector (which includes both th, th0)
def svm_obj_grad(X, y, th, th0, lam):
    d_th = d_svm_obj_th(X, y, th, th0, lam)
    d_th0 = d_svm_obj_th0(X, y, th, th0, lam)
    return np.vstack((d_th, d_th0))

[[-0.06], [0.3], [0.0]]

# Batch Gradient Descent for SVM

In [138]:
def batch_svm_min(data, labels, lam):
    d, n = data.shape
    def f(x):
        th = np.array([x[:-1, 0]]).T
        th0 = np.array([[x[-1, 0]]])
        return svm_obj(data, labels, th, th0, lam)
    def df(x):
        th = np.array([x[:-1, 0]]).T
        th0 = np.array([[x[-1, 0]]])
        return svm_obj_grad(data, labels, th, th0, lam)
    def svm_min_step_size_fn(i):
        return 2/(i+1)**0.5
    return gd(f, df, np.zeros((d+1, 1)), svm_min_step_size_fn, 10)